In [ ]:

import ipywidgets as widgets
from IPython.display import display, clear_output
import json
from datetime import datetime
from functools import partial
import os

todo_items = []

layout_medium = widgets.Layout(width='130px')
layout_small = widgets.Layout(width='70px')
layout_button = widgets.Layout(width='60px', height='32px')
layout_action = widgets.Layout(width='140px', height='32px')
layout_input = widgets.Layout(width='240px')

overall_progress_bar = widgets.FloatProgress(min=0, max=100, value=0, bar_style='', layout=widgets.Layout(width='40%', height='24px'), style={'bar_color': 'black'})
overall_progress_label = widgets.Label(value='(0%)', layout=widgets.Layout(margin='0 10px'))

todo_list_box = widgets.VBox(layout=widgets.Layout(border='1px solid #ccc', padding='10px'))

def update_overall_progress():
    if not todo_items:
        overall_progress_bar.value = 0
        overall_progress_label.value = "(0%)"
        return
    avg = sum(i['progress'] for i in todo_items) / len(todo_items)
    overall_progress_bar.value = avg
    overall_progress_label.value = f"({int(avg)}%)"

def refresh_list():
    todo_list_box.children = []

    headers = ['할 일', '진행률 + 입력', '마감일', '중요도', '작업']
    header_widgets = [widgets.Label(value=h, layout=layout_input if h == '할 일' else layout_medium) for h in headers]
    todo_list_box.children = [widgets.HBox(header_widgets)]

    for item in todo_items:
        title = widgets.Text(value=item['title'], layout=layout_input)
        progress_bar = widgets.FloatProgress(value=item['progress'], min=0, max=100, layout=widgets.Layout(width='100%', height='20px'))
        progress_slider = widgets.IntSlider(value=item['progress'], min=0, max=100, step=5, layout=widgets.Layout(width='100%'))
        progress_col = widgets.VBox([progress_bar, progress_slider], layout=layout_medium)
        due_label = widgets.Label(value=item['due'].strftime('%Y-%m-%d') if item['due'] else '-', layout=layout_medium)
        priority_label = widgets.Label(value=item['priority'], layout=layout_medium)

        complete_btn = widgets.Button(description='완료 ✅', layout=layout_button, style={'button_color': '#b8b8b8'})
        delete_btn = widgets.Button(description='삭제 ❌', layout=layout_button, style={'button_color': '#b8b8b8'})
        button_box = widgets.HBox([complete_btn, delete_btn], layout=layout_action)

        def on_progress_change(change, i=item, bar=progress_bar):
            i['progress'] = change['new']
            bar.value = change['new']
            update_overall_progress()

        def on_complete_clicked(b, i=item):
            i['progress'] = 100
            update_overall_progress()
            refresh_list()

        def on_delete_clicked(b, i=item):
            todo_items.remove(i)
            update_overall_progress()
            refresh_list()

        progress_slider.observe(on_progress_change, names='value')
        complete_btn.on_click(partial(on_complete_clicked, i=item))
        delete_btn.on_click(partial(on_delete_clicked, i=item))

        row = widgets.HBox([title, progress_col, due_label, priority_label, button_box])
        todo_list_box.children += (row,)


In [ ]:

title_input = widgets.Text(placeholder='할 일을 입력하세요', layout=layout_input)
progress_input = widgets.IntSlider(value=0, min=0, max=100, step=5, layout=layout_medium)
due_input = widgets.DatePicker(layout=layout_medium)
priority_input = widgets.Dropdown(options=['낮음', '보통', '높음'], value='보통', layout=layout_medium)
add_button = widgets.Button(description='추가', button_style='success', layout=widgets.Layout(width='280px', height='32px'))

def add_task(b):
    if title_input.value.strip():
        todo_items.append({
            'title': title_input.value.strip(),
            'progress': progress_input.value,
            'due': due_input.value,
            'priority': priority_input.value
        })
        title_input.value = ''
        progress_input.value = 0
        due_input.value = None
        priority_input.value = '보통'
        update_overall_progress()
        refresh_list()

add_button.on_click(add_task)


In [ ]:

def save_data(b):
    with open('todo_data.json', 'w') as f:
        json.dump([
            {
                **i,
                'due': i['due'].strftime('%Y-%m-%d') if i['due'] else None
            } for i in todo_items
        ], f)

def load_data(b):
    try:
        if os.path.exists('todo_data.json'):
            with open('todo_data.json', 'r') as f:
                raw = json.load(f)
                todo_items.clear()
                for i in raw:
                    todo_items.append({
                        'title': i['title'],
                        'progress': i['progress'],
                        'due': datetime.strptime(i['due'], '%Y-%m-%d').date() if i['due'] else None,
                        'priority': i.get('priority', '보통')
                    })
                update_overall_progress()
                refresh_list()
    except:
        print("불러오기 실패")

save_btn = widgets.Button(description='저장', button_style='info', layout=layout_medium)
load_btn = widgets.Button(description='불러오기', button_style='warning', layout=layout_medium)
save_btn.on_click(save_data)
load_btn.on_click(load_data)


In [ ]:

header_label = widgets.Label(value='DAVID mk17 - UI 개선 및 오류 수정')

row1 = widgets.HBox([widgets.Label('할 일', layout=layout_small), title_input])
row2 = widgets.HBox([widgets.Label('진행률', layout=layout_small), progress_input])
row3 = widgets.HBox([widgets.Label('마감일', layout=layout_small), due_input])
row4 = widgets.HBox([widgets.Label('중요도', layout=layout_small), priority_input])
row5 = widgets.HBox([add_button], layout=widgets.Layout(justify_content='center'))
row6 = widgets.HBox([save_btn, load_btn], layout=widgets.Layout(justify_content='center'))
row7 = widgets.HBox([overall_progress_bar, overall_progress_label], layout=widgets.Layout(justify_content='center'))

display(widgets.VBox([
    header_label,
    row1,
    row2,
    row3,
    row4,
    row5,
    row6,
    row7,
    todo_list_box
]))

update_overall_progress()
refresh_list()
